In [19]:
import os
import pandas as pd
import zipfile
import pyodbc
import requests
import io
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import sqlalchemy
import urllib
import warnings
warnings.filterwarnings('ignore')

In [20]:
def iterar_entre_meses(fecha_desde, fecha_hasta):
    fecha_actual = datetime.strptime(fecha_desde, "%Y-%m-%dT%H:%M:%S.%f%z")
    fecha_fin = datetime.strptime(fecha_hasta, "%Y-%m-%dT%H:%M:%S.%f%z")
    
    while fecha_actual < fecha_fin:
        # Primer día del mes actual
        primer_dia_mes = fecha_actual.replace(day=1)
        # Último día del mes actual
        if fecha_actual.month == 12:
            ultimo_dia_mes = fecha_actual.replace(day=31)
        else:
            primer_dia_siguiente_mes = fecha_actual.replace(day=1) + relativedelta(months=1)
            ultimo_dia_mes = primer_dia_siguiente_mes - timedelta(days=1)
        
        yield primer_dia_mes, ultimo_dia_mes
        
        # Avanza al siguiente mes
        fecha_actual = fecha_actual + relativedelta(months=1)

In [21]:
for fecha_actual, fecha_siguiente in iterar_entre_meses("2023-01-01T00:00:00.000-03:00", "2023-01-31T00:00:00.000-03:00"):
    print(fecha_actual)
    print(fecha_siguiente)
    mes_mdb = fecha_actual.strftime("%m-%Y")
    print(mes_mdb)

2023-01-01 00:00:00-03:00
2023-01-31 00:00:00-03:00
01-2023


In [38]:
NEMO = "DTE_EMISION"

URL = f"https://api.cammesa.com/pub-svc/public/"

server = 'DARCCVWSQL19'
database = 'TAPI'

tabla_Q_Reporte_NO_Corregidos = 'Q_Reporte_NO_Corregidos'
tabla_GenTER_NO_Corregidos = 'GenTER_NO_Corregidos'
tabla_GenHID_NO_Corregidos = 'GenHID_NO_Corregidos'
tabla_GenMATER_NO_Corregidos = 'GenMATER_NO_Corregidos'
tabla_GenBOM_NO_Corregidos = 'GenBOM_NO_Corregidos'

tabla_Q_Reporte_Corregidos = 'Q_Reporte_Corregidos'
tabla_GenTER_Corregidos = 'GenTER'
tabla_GenHID_Corregidos = 'GenHID'
tabla_GenMATER_Corregidos = 'GenMATER'
tabla_GenBOM_Corregidos = 'GenBOM'

method_id = "findDocumentosByNemoRango?" #ID
method_zip = "findAllAttachmentZipByNemoId?"

fecha_desde = "2023-03-01T00:00:00.000-03:00"
fecha_hasta = "2024-05-31T23:59:59.000-03:00"
fecha_desde_obj = 0
fecha_hasta_obj = 1

connection_string = f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};Trusted_Connection=yes;'

In [40]:
def DTE_corregidos():

    if fecha_desde_obj < fecha_hasta_obj:
        print("Se realiza el update DTE")
        
        for fecha_actual, fecha_siguiente in iterar_entre_meses(fecha_desde,fecha_hasta):

            url_doc_id = f"{URL}{method_id}fechadesde={fecha_actual.isoformat()}&fechahasta={fecha_siguiente.isoformat()}&nemo={NEMO}"

            mes_mdb = fecha_actual.strftime("%m-%Y")
            print(mes_mdb)

            try:
                with requests.get(url_doc_id) as response:
                    if response.status_code == 200:
                        DTE=response.json()
                        doc_id = DTE[-1]['id']

                    else:
                        print("La solicitud falló con el código de estado:", response.status_code)
            except requests.exceptions.RequestException as e:
                # Manejar la excepción
                print("Error al realizar la solicitud:", e)
            
            url_zip = f"{URL}{method_zip}docId={doc_id}&nemo={NEMO}"

            try:
                with requests.get(url_zip) as r:

                    if r.status_code == 200: #response 200
                        with zipfile.ZipFile(io.BytesIO(r.content)) as primer_zip: #trabaja el primer script
                            
                            files_inside = primer_zip.namelist() #listas archivos del primer zip
                            DTE_name = primer_zip.namelist()[0]
                            DTE_fecha = primer_zip.namelist()[0][:7]
                            
                            #print(DTE_fecha)

                            path_dte = f"../.zips/{DTE_fecha}"   
                            
                            primer_zip.extract(f"{DTE_name}", path= path_dte)

                            with zipfile.ZipFile(f"{path_dte}/{DTE_name}", 'r') as segundo_zip: #trabaja el segundo script

                                DTE_mdb_name = f"{DTE_fecha}.MDB"         
                                
                                segundo_zip.extract(f"{DTE_mdb_name}", path=path_dte)

                                mdb_file = os.path.join(path_dte, DTE_mdb_name)
                                conn_str = f"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={mdb_file};"

                                conn = pyodbc.connect(conn_str)

                                Q_Reporte = pd.read_sql("SELECT * FROM Q_Reporte", conn)
                                GenMATER = pd.read_sql("SELECT * FROM GenMATER", conn)
                                GenTER = pd.read_sql("SELECT * FROM GenTER", conn)
                                GenHID = pd.read_sql("SELECT * FROM GenHID", conn)
                                GenBOM = pd.read_sql("SELECT * FROM GenBOM", conn)
                                
                                conn.close()

                                #print(GenTER.columns)
                                mes_datetime = datetime.strptime(mes_mdb, '%m-%Y')
                                mes_mdb_formated = mes_datetime.strftime('%Y-%m')

                                Q_Reporte.insert(0, 'FECHA', mes_mdb_formated)
                                GenMATER.insert(0, 'FECHA', mes_mdb_formated)
                                GenTER.insert(0, 'FECHA', mes_mdb_formated)
                                GenHID.insert(0, 'FECHA', mes_mdb_formated)
                                GenBOM.insert(0, 'FECHA', mes_mdb_formated)

                                quoted = urllib.parse.quote_plus(connection_string)

                                agentes = ["CPBUENAG", "CTBARRAG","CTGUEMEG","CTGUEPLG","CTLLATAG","CTPBUEMG","CTPIQUIG",
                                           "CTPPILAG","ECOENERG","GENELBAG","GENELCCG","GENELPLG","HIDIAMAG","HINIHUIG",
                                           "HREYESYG","PAMPAENG","PEPAMPAG","PEVARAUG","PPLEUFUG"]

                                df_Q_Reporte = Q_Reporte[Q_Reporte["NEMO"].isin(agentes)]
                                df_GenMATER = GenMATER[GenMATER["AGENTE"].isin(agentes)]
                                df_GenTER = GenTER[GenTER["AGENTE"].isin(agentes)]
                                df_GenHID = GenHID[GenHID["AGENTE"].isin(agentes)]
                                df_GenBOM = GenBOM[GenBOM["AGENTE"].isin(agentes)]

                                engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

                                df_Q_Reporte.to_sql(f'{tabla_Q_Reporte_Corregidos}', schema='dbo', con=engine, if_exists='append', chunksize=20000)
                                df_GenMATER.to_sql(f'{tabla_GenMATER_Corregidos}', schema='dbo', con=engine, if_exists='append', chunksize=20000)
                                df_GenTER.to_sql(f'{tabla_GenTER_Corregidos}', schema='dbo', con=engine, if_exists='append', chunksize=20000)
                                df_GenHID.to_sql(f'{tabla_GenHID_Corregidos}', schema='dbo', con=engine, if_exists='append', chunksize=20000)
                                df_GenBOM.to_sql(f'{tabla_GenBOM_Corregidos}', schema='dbo', con=engine, if_exists='append', chunksize=20000)
                        
                                #Q_Reporte.to_sql(f'{tabla_Q_Reporte_Corregidos}', schema='dbo', con=engine, if_exists='append', chunksize=20000)
                                #GenMATER.to_sql(f'{tabla_GenMATER_Corregidos}', schema='dbo', con=engine, if_exists='append', chunksize=20000)
                                #GenTER.to_sql(f'{tabla_GenTER_Corregidos}', schema='dbo', con=engine, if_exists='append', chunksize=20000)
                                #GenHID.to_sql(f'{tabla_GenHID_Corregidos}', schema='dbo', con=engine, if_exists='append', chunksize=20000)
                                #GenBOM.to_sql(f'{tabla_GenBOM_Corregidos}', schema='dbo', con=engine, if_exists='append', chunksize=20000)
                            
                    else:
                        print("La solicitud falló con el código de estado:", response.status_code)

            except requests.exceptions.RequestException as e:
                # Manejar la excepción
                print("Error al realizar la solicitud:", e)

In [41]:
DTE_corregidos()

Se realiza el update DTE
03-2023
04-2023
05-2023
06-2023
07-2023
08-2023
09-2023
10-2023
11-2023
12-2023
01-2024
02-2024
03-2024
04-2024
05-2024
